In [1]:
#importing libraries
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter
import scipy
import json
import torchvision.transforms.functional as F
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
from tqdm import tqdm
from natsort import natsorted

%matplotlib inline

In [2]:
from torchvision import datasets, transforms
transform=transforms.Compose([
                      transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225]),
                  ])

In [3]:
root = "test/test_images/circles-test"

In [4]:
easy = os.path.join(root,"easy")
real = os.path.join(root,"realistic")
challanging = os.path.join(root,"challanging")

In [5]:
img_paths = []
for img_path in natsorted(glob.glob(os.path.join(easy, "*.jpg"))):
    img_paths.append(img_path)

In [ ]:
model = CSRNet()
model = model.cuda()
checkpoint = torch.load('test/circles/38checkpoint.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

In [73]:
from matplotlib import cm as c
folder = real
n = 1
mae = 0
total = 0
under = 0
over = 0
for i in img_paths:
    fig = plt.figure(figsize=(20, 20))
    image = Image.open(i).convert('RGB')
    image = image.resize((1024, 768))
    img = transform(image.convert('RGB')).cuda()
    output = model(img.unsqueeze(0))
    #print("Predicted Count : ",np.round(output.detach().cpu().sum().numpy(), 3))
    predicted = np.round(output.detach().cpu().sum().numpy(), 3)
    temp = np.asarray(output.detach().cpu().reshape(output.detach().cpu().shape[2],output.detach().cpu().shape[3]))
    plt.subplot(2, 1, 1)
    plt.title(str(predicted))
    plt.imshow(temp,cmap = c.jet)
    
    original_count = int((i.split('/')[-1]).split('.')[0])
    total += original_count
    
    error = predicted - original_count
    if error > 0:
        over += error
    elif error <0:
        under += abs(error)
    mae += abs(error)
    
    
    original_image = plt.imread(i)
    plt.subplot(2, 1, 2)
    plt.title('Original Count ' + str(original_count))
    plt.imshow(original_image)
    

    
    plt.savefig(folder+'test'+str(n)+'.jpg', bbox_inches='tight')
    
    plt.close()
    n = n+1
    

print("Out of {} objects {} objects were miss-counted. {} were overcounted and {} were undercounted"
              .format(np.round(total, 3), np.round(mae, 3), np.round(over,3), np.round(under,3)))
print("Mean absolute error ", np.round(mae/9, 3))

Out of 399 objects 22.211 objects were miss-counted. 6.487 were overcounted and 15.724 were undercounted
Mean absolute error  2.468
